In [267]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [268]:
import numpy as np
import pandas as pd
import scipy as sp
import mne
import keras
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

from torch import optim
import argparse
from random import random
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
import torch.nn.functional as F
from os import walk

In [269]:
DEVICE = torch.device('cuda:0')

In [270]:
def prepad0(data, seglength):
    pad = (-len(data)) % seglength
    l = np.concatenate((data,np.zeros((pad, len(data[0])))))
    return l

In [271]:
def pad(rawdata, seglength):
    pad_length = -rawdata.size()[0] % seglength
    padded = torch.cat((rawdata, torch.zeros((pad_length, rawdata.size()[1]))), 0)
    return padded

In [272]:
def to_epochs(rawdata, seg_length = 9000):
    data = pad(rawdata, seg_length)
    return torch.split(data, seg_length)

In [273]:
def to_gpu(x, on_cpu=False, gpu_id=None):

    if torch.cuda.is_available() and not on_cpu:
        x = x.cuda(gpu_id)
    return x

In [274]:
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(d, device) for d in data]
    return data.to(device, non_blocking=True)

In [275]:
def generate_batches(files, batch_size, INPUT_DATA_DIR):
    counter = 0

    while True:
        fname = files[counter]

        counter = (counter + 1) % len(files)
        frame = torch.Tensor(pd.read_csv(INPUT_DATA_DIR + f'\{fname}', skiprows=2, header=None).values)

        # here is your preprocessing

        input = frame
        output = torch.Tensor(1)

        for local_index in range(0, input.shape[0], batch_size):
            input_local = input[local_index:(local_index + batch_size)]
            output_local = output[local_index:(local_index + batch_size)]

            yield input_local, output_local

In [276]:
"""
seg_length = 400
ige = to_epochs(raw_ige, seg_length)
focal = to_epochs(raw_focal, seg_length)
ige_labels = torch.zeros((len(ige)))
focal_labels = torch.ones((len(focal)))
"""

'\nseg_length = 400\nige = to_epochs(raw_ige, seg_length)\nfocal = to_epochs(raw_focal, seg_length)\nige_labels = torch.zeros((len(ige)))\nfocal_labels = torch.ones((len(focal)))\n'

In [277]:
"""
INPUT_DATA_DIR = 'D:\EEG\TLE'
TRAIN_DATA_COEFFICIENT = 0.75

files = []
for (dirpath, dirnames, filenames) in walk(INPUT_DATA_DIR):
    files.extend(filenames)
    break

train_files_finish = int(len(files) * TRAIN_DATA_COEFFICIENT)
train_files = files[0:train_files_finish]
validation_files = files[train_files_finish:len(files)]
"""

"\nINPUT_DATA_DIR = 'D:\\EEG\\TLE'\nTRAIN_DATA_COEFFICIENT = 0.75\n\nfiles = []\nfor (dirpath, dirnames, filenames) in walk(INPUT_DATA_DIR):\n    files.extend(filenames)\n    break\n\ntrain_files_finish = int(len(files) * TRAIN_DATA_COEFFICIENT)\ntrain_files = files[0:train_files_finish]\nvalidation_files = files[train_files_finish:len(files)]\n"

In [278]:
"""
batch_size = 1000 
train_generator = generate_batches(files=train_files, batch_size=batch_size, INPUT_DATA_DIR = 'D:\EEG\TLE')
test_generator = generate_batches(files=validation_files, batch_size=batch_size, INPUT_DATA_DIR = 'D:\EEG\TLE')
"""

"\nbatch_size = 1000 \ntrain_generator = generate_batches(files=train_files, batch_size=batch_size, INPUT_DATA_DIR = 'D:\\EEG\\TLE')\ntest_generator = generate_batches(files=validation_files, batch_size=batch_size, INPUT_DATA_DIR = 'D:\\EEG\\TLE')\n"

In [279]:
names = []
for path, _, name in os.walk('D:\EEG\PNES'):
    names.append(name)
    names = np.squeeze(names)


In [280]:
data = torch.Tensor(pd.read_csv(f"{path}\{names[0]}" , skiprows=2, header=None).values)

In [281]:
pnes = []
for i in range(3):
    pnes.append(torch.Tensor(pd.read_csv(f"{path}\{names[i]}" , skiprows=2, header=None).values))